<a href="https://colab.research.google.com/github/lokeshtalamala1/Deep_Learning_Assignment_1/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#initializing my project and entity in wandb using its atributes
#Deep_Learning_Assignment_1
#cs24m023-indian-institute-of-technology-madras

!pip install wandb tensorflow
import wandb
wandb.init(project='Deep_Learning_Assignment_1', entity='cs24m023-indian-institute-of-technology-madras')


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs24m023 (cs24m023-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
#import numpy ,fashion_mnist and load data

from keras.datasets import fashion_mnist
import numpy as np

((x_train,y_train),(x_test,y_test)) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
import matplotlib.pyplot as plt

In [4]:
#example image from each class from given data

#QUESTION 1

images = []
classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
for class_label in range(len(classes)):
    index = next(i for i, y in enumerate(y_train) if y == class_label)
    images.append(wandb.Image(x_train[index], caption=classes[class_label]))

wandb.log({"example image from each class": images})
plt.show()
wandb.finish()

In [5]:
#QUESTION 2 AND QUESTION 3
#creating a class with a layer as an object and parametes to the layer are :
    #input dimensions
    #activation_function
    #optimizer_function
    #weight initialization

#FORWARD PROPAGATE AND BACKWARD PROPAGATE
#feedforwardneuralnetwork for calculating all the layers over the model and loss functions are calculated
#with input parameters optimizer,activation function weight initialization
#number of epochs and size of each layer ,learnig rate theeta

import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

# -------------------- FeedForwardNN Class --------------------
class FeedForwardNN:
    def __init__(self, input_size=None, hidden_nodes=None, output_size=None, layer_dims=None,
                 activation_hidden='relu', init_type='xavier'):
        """
        Initializes the neural network.

        You can specify the architecture in two ways:
          1. Directly pass a list of layer dimensions as 'layer_dims'.
          2. Or pass the input size, list of hidden layer node counts, and output size.
        """
        if layer_dims is not None:
            self.layer_dims = layer_dims
        else:
            if input_size is None or hidden_nodes is None or output_size is None:
                raise ValueError("Either layer_dims or input_size, hidden_nodes, and output_size must be provided")
            self.layer_dims = [input_size] + hidden_nodes + [output_size]

        self.L = len(self.layer_dims) - 1  # Number of layers excluding the input layer
        self.init_type = init_type
        self.activation_hidden = activation_hidden
        self.parameters = self.initialize_parameters(self.layer_dims, init_type)

    # ---- Activation Functions and Their Gradients ----
    def sigmoid(z):
        return 1.0 / (1.0 + np.exp(-z))

    def dsigmoid(z):
        s = FeedForwardNN.sigmoid(z)
        return s * (1 - s)

    def tanh(z):
        return np.tanh(z)

    def dtanh(z):
        return 1 - np.tanh(z)**2

    def relu(z):
        return np.maximum(0, z)

    def drelu(z):
        return np.array(z > 0, dtype=float)

    def softmax(z):
        z_exp = np.exp(z - np.max(z, axis=0, keepdims=True))
        return z_exp / np.sum(z_exp, axis=0, keepdims=True)

    def dsoftmax(z):
        s = FeedForwardNN.softmax(z.reshape(-1, 1))
        jacobian = np.diagflat(s) - np.dot(s, s.T)
        return jacobian

    def get_activation_function(self):
        if self.activation_hidden.lower() == 'sigmoid':
            return FeedForwardNN.sigmoid, FeedForwardNN.dsigmoid
        elif self.activation_hidden.lower() == 'tanh':
            return FeedForwardNN.tanh, FeedForwardNN.dtanh
        elif self.activation_hidden.lower() == 'relu':
            return FeedForwardNN.relu, FeedForwardNN.drelu
        else:
            raise ValueError("Unsupported activation function. Choose from 'sigmoid', 'tanh', or 'relu'.")

    # ---- Parameter Initialization ----
    def initialize_parameters(self, layer_dims, init_type='random'):
        parameters = {}
        L = len(layer_dims)
        for l in range(1, L):
            if init_type.lower() == 'random':
                parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1])
            elif init_type.lower() == 'xavier':
                parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * np.sqrt(1.0 / layer_dims[l-1])
            else:
                raise ValueError("init_type must be 'random' or 'xavier'")
            parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        return parameters

    # ---- Forward Propagation ----
    def forward_propagation(self, X):
        caches = []
        A = X
        activation_func, _ = self.get_activation_function()
        # Process hidden layers
        for l in range(1, self.L):
            W = self.parameters['W' + str(l)]
            b = self.parameters['b' + str(l)]
            Z = np.dot(W, A) + b
            A = activation_func(Z)
            caches.append((Z, A))
        # Output layer with softmax activation
        W = self.parameters['W' + str(self.L)]
        b = self.parameters['b' + str(self.L)]
        Z = np.dot(W, A) + b
        A_final = FeedForwardNN.softmax(Z)
        caches.append((Z, A_final))
        return A_final, caches

    # ---- Backpropagation ----
    def back_propagation(self, X, Y, caches, weight_decay=0.0):
        grads = {}
        m = X.shape[1]
        L = self.L
        _, activation_grad = self.get_activation_function()
        # Output layer gradient (using softmax + cross-entropy)
        ZL, AL = caches[-1]
        dZL = AL - Y
        WL_prev = self.parameters['W' + str(L)]
        A_prev = caches[-2][1] if L > 1 else X
        grads["dW" + str(L)] = (np.dot(dZL, A_prev.T) / m) + weight_decay * self.parameters['W' + str(L)]
        grads["db" + str(L)] = np.sum(dZL, axis=1, keepdims=True) / m
        dA_prev = np.dot(WL_prev.T, dZL)
        # Backprop for hidden layers
        for l in reversed(range(1, L)):
            Z, A = caches[l-1]
            dZ = dA_prev * activation_grad(Z)
            A_prev = X if l == 1 else caches[l-2][1]
            grads["dW" + str(l)] = (np.dot(dZ, A_prev.T) / m) + weight_decay * self.parameters['W' + str(l)]
            grads["db" + str(l)] = np.sum(dZ, axis=1, keepdims=True) / m
            if l > 1:
                W_prev = self.parameters['W' + str(l)]
                dA_prev = np.dot(W_prev.T, dZ)
        return grads

    # ---- Compute Loss and Accuracy ----
    def compute_loss_and_accuracy(self, X, Y):
        A, _ = self.forward_propagation(X)
        m = X.shape[1]
        loss = -np.sum(Y * np.log(A + 1e-8)) / m
        predictions = np.argmax(A, axis=0)
        labels = np.argmax(Y, axis=0)
        accuracy = np.mean(predictions == labels)
        return loss, accuracy

    # ---- Training Step (Mini-Batch) ----
    def train_step(self, X_batch, Y_batch, optimizer, weight_decay, t=1):
        A_final, caches = self.forward_propagation(X_batch)
        m = X_batch.shape[1]
        loss = -np.sum(Y_batch * np.log(A_final + 1e-8)) / m
        grads = self.back_propagation(X_batch, Y_batch, caches, weight_decay=weight_decay)
        self.parameters = optimizer.update_parameters(self.parameters, grads, t)
        return loss

    # ---- Training Loop with Mini-Batches ----
    def train(self, X_train, Y_train, X_val, Y_val, optimizer, epochs=10, batch_size=32, weight_decay=0.0):
        m = X_train.shape[1]
        t = 1  # time step for optimizers (e.g., Adam)
        for epoch in range(1, epochs + 1):
            permutation = np.random.permutation(m)
            X_shuffled = X_train[:, permutation]
            Y_shuffled = Y_train[:, permutation]
            epoch_loss = 0
            num_batches = m // batch_size
            for i in range(num_batches):
                start = i * batch_size
                end = start + batch_size
                X_batch = X_shuffled[:, start:end]
                Y_batch = Y_shuffled[:, start:end]
                loss = self.train_step(X_batch, Y_batch, optimizer, weight_decay, t)
                epoch_loss += loss
                t += 1
            epoch_loss /= num_batches
            train_loss, train_acc = self.compute_loss_and_accuracy(X_train, Y_train)
            val_loss, val_acc = self.compute_loss_and_accuracy(X_val, Y_val)
            print(f"Epoch {epoch:03d}: Epoch Loss = {epoch_loss:.4f} | Train Loss = {train_loss:.4f}, Train Acc = {train_acc:.4f} | Val Loss = {val_loss:.4f}, Val Acc = {val_acc:.4f}")
        # Optionally, metrics can be returned or logged externally.


# -------------------- Optimizers Class --------------------
class Optimizers:
    def __init__(self, method='sgd', learning_rate=0.01, momentum=0.9, beta1=0.9, beta2=0.999, epsilon=1e-8, decay=0.0):
        """
        Initializes the optimizer.

        Supported methods: 'sgd', 'momentum', 'nesterov', 'rmsprop', 'adam', 'nadam'
        """
        self.method = method.lower()
        self.lr = learning_rate
        self.momentum = momentum
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.decay = decay
        self.v = {}
        self.s = {}

    def initialize_state(self, parameters):
        for key in parameters:
            self.v[key] = np.zeros_like(parameters[key])
            self.s[key] = np.zeros_like(parameters[key])

    def update_parameters(self, parameters, grads, t=1):
        if not self.v or not self.s:
            self.initialize_state(parameters)
        updated_parameters = {}
        for key in parameters.keys():
            grad = grads.get("d" + key, None)
            if grad is None:
                updated_parameters[key] = parameters[key]
                continue
            if self.method == 'sgd':
                update = self.lr * grad
            elif self.method == 'momentum':
                self.v[key] = self.momentum * self.v[key] + self.lr * grad
                update = self.v[key]
            elif self.method == 'nesterov':
                v_prev = self.v[key].copy()
                self.v[key] = self.momentum * self.v[key] + self.lr * grad
                update = self.momentum * v_prev + (1 + self.momentum) * self.lr * grad
            elif self.method == 'rmsprop':
                self.s[key] = self.beta2 * self.s[key] + (1 - self.beta2) * (grad ** 2)
                update = self.lr * grad / (np.sqrt(self.s[key]) + self.epsilon)
            elif self.method == 'adam':
                self.v[key] = self.beta1 * self.v[key] + (1 - self.beta1) * grad
                self.s[key] = self.beta2 * self.s[key] + (1 - self.beta2) * (grad ** 2)
                v_corrected = self.v[key] / (1 - self.beta1**t)
                s_corrected = self.s[key] / (1 - self.beta2**t)
                update = self.lr * v_corrected / (np.sqrt(s_corrected) + self.epsilon)
            elif self.method == 'nadam':
                self.v[key] = self.beta1 * self.v[key] + (1 - self.beta1) * grad
                self.s[key] = self.beta2 * self.s[key] + (1 - self.beta2) * (grad ** 2)
                v_corrected = self.v[key] / (1 - self.beta1**t)
                s_corrected = self.s[key] / (1 - self.beta2**t)
                update = self.lr * (self.beta1 * v_corrected + (1 - self.beta1) * grad / (1 - self.beta1**t)) / (np.sqrt(s_corrected) + self.epsilon)
            else:
                raise ValueError("Unsupported optimization method.")
            updated_parameters[key] = parameters[key] - update
        return updated_parameters


# ---- Utility: Load and Preprocess Fashion MNIST ----
def load_preprocess_fashion_mnist():
    (X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
    # Normalize and flatten images: shape -> (784, num_examples)
    X_train_full = X_train_full.reshape(X_train_full.shape[0], -1).T / 255.0
    X_test = X_test.reshape(X_test.shape[0], -1).T / 255.0
    # Convert labels to one-hot encoding: shape -> (10, num_examples)
    Y_train_full = to_categorical(y_train_full, num_classes=10).T
    Y_test = to_categorical(y_test, num_classes=10).T
    # Split training into 90% train and 10% validation
    m = X_train_full.shape[1]
    split_index = int(0.9 * m)
    X_train = X_train_full[:, :split_index]
    Y_train = Y_train_full[:, :split_index]
    X_val = X_train_full[:, split_index:]
    Y_val = Y_train_full[:, split_index:]
    return X_train, Y_train, X_val, Y_val, X_test, Y_test


In [6]:
#SWEEP WITH BAYESIAN
#QUESTION 4 AND 5 AND 6 SWEEP

# -------------------- Sweep Configuration --------------------
sweep_config = {
    "method": "bayes",  # Bayesian strategy is chosen for efficient search in a large hyperparameter space.
    "metric": {
        "name": "val_accuracy",
        "goal": "maximize"
    },
    "parameters": {
        "epochs": {"values": [5, 10]},
        "num_hidden_layers": {"values": [3, 4, 5]},
        "hidden_size": {"values": [32, 64, 128]},
        "weight_decay": {"values": [0, 0.0005, 0.5]},
        "learning_rate": {"values": [1e-3, 1e-4]},
        "optimizer": {"values": ["sgd", "momentum", "nesterov", "rmsprop", "adam", "nadam"]},
        "batch_size": {"values": [16, 32, 64]},
        "weight_initialisation": {"values": ["random", "xavier"]},
        "activation_function": {"values": ["sigmoid", "tanh", "relu"]}
    }
}


In [9]:

#QUESTION 4 AND 5 AND 6 MAKING FUNCTION CALL
#defining train function for wandb agent to run it's configurations (directly from documentation)

import wandb

# -------------------- Sweep Agent Function --------------------
def sweep_train():
    # Initialize a wandb run so that config is available
    run = wandb.init()
    config = wandb.config

    # Set a meaningful name for the run using key hyperparameters:
    # Example: hl_3_bs_16_ac_tanh (3 hidden layers, batch size 16, activation function tanh)
    run.name = f"hl_{config.num_hidden_layers}_bs_{config.batch_size}_ac_{config.activation_function}"

    # Load the Fashion MNIST data (90% train, 10% validation, test split remains)
    X_train, Y_train, X_val, Y_val, X_test, Y_test = load_preprocess_fashion_mnist()

    # Construct network architecture from the sweep config:
    layer_dims = [784] + [config.hidden_size] * config.num_hidden_layers + [10]

    # Create the network instance with the given activation function and weight initialization
    nn = FeedForwardNN(layer_dims=layer_dims,
                       activation_hidden=config.activation_function,
                       init_type=config.weight_initialisation)

    # Create optimizer instance based on the config values
    optimizer = Optimizers(method=config.optimizer, learning_rate=config.learning_rate)

    # Train the network with the specified epochs, batch size, and weight decay
    nn.train(X_train, Y_train, X_val, Y_val, optimizer,
             epochs=config.epochs, batch_size=config.batch_size, weight_decay=config.weight_decay)

    # After training, log final performance on the test set
    test_loss, test_acc = nn.compute_loss_and_accuracy(X_test, Y_test)
    print(f"Test Loss = {test_loss:.4f}, Test Accuracy = {test_acc:.4f}")
    wandb.log({"test_loss": test_loss, "test_accuracy": test_acc})

    # Properly finish the wandb run
    wandb.finish()

if __name__ == "__main__":
    # Ensure you're logged in to wandb (will prompt for API key if necessary)
    wandb.login()

    # Initialize a sweep with the above configuration; project name can be set as desired.
    sweep_id = wandb.sweep(sweep_config, project="Deep_Learning_Assignment_1")

    # Run the sweep agent; adjust 'count' to control the number of runs.
    wandb.agent(sweep_id, function=sweep_train, count=100)


Create sweep with ID: f576eipe
Sweep URL: https://wandb.ai/cs24m023-indian-institute-of-technology-madras/Deep_Learning_Assignment_1/sweeps/f576eipe


wandb: Agent Starting Run: yvx92scl with config:
wandb: 	activation_function: tanh
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	hidden_size: 128
wandb: 	learning_rate: 0.0001
wandb: 	num_hidden_layers: 5
wandb: 	optimizer: rmsprop
wandb: 	weight_decay: 0.5
wandb: 	weight_initialisation: random


Epoch 001: Epoch Loss = 9.7953 | Train Loss = 7.6332, Train Acc = 0.1752 | Val Loss = 7.6630, Val Acc = 0.1785
Epoch 002: Epoch Loss = 5.9664 | Train Loss = 4.3629, Train Acc = 0.2749 | Val Loss = 4.3602, Val Acc = 0.2653
Epoch 003: Epoch Loss = 3.3171 | Train Loss = 2.3681, Train Acc = 0.4014 | Val Loss = 2.3633, Val Acc = 0.4077
Epoch 004: Epoch Loss = 1.8031 | Train Loss = 1.3499, Train Acc = 0.5464 | Val Loss = 1.3546, Val Acc = 0.5485
Epoch 005: Epoch Loss = 1.1500 | Train Loss = 1.0292, Train Acc = 0.6709 | Val Loss = 1.0293, Val Acc = 0.6712
Test Loss = 1.0511, Test Accuracy = 0.6600


test_accuracy,▁
test_loss,▁
test_accuracy,0.66
test_loss,1.05106


wandb: Agent Starting Run: 5iiidnvz with config:
wandb: 	activation_function: relu
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.0001
wandb: 	num_hidden_layers: 5
wandb: 	optimizer: nadam
wandb: 	weight_decay: 0.5
wandb: 	weight_initialisation: xavier


...

Epoch 001: Epoch Loss = 2.5046 | Train Loss = 1.3792, Train Acc = 0.5229 | Val Loss = 1.4052, Val Acc = 0.5188
Epoch 002: Epoch Loss = 1.2533 | Train Loss = 1.1272, Train Acc = 0.5664 | Val Loss = 1.1144, Val Acc = 0.5717
Epoch 003: Epoch Loss = 1.0534 | Train Loss = 0.9652, Train Acc = 0.6317 | Val Loss = 0.9606, Val Acc = 0.6370
Epoch 004: Epoch Loss = 0.9643 | Train Loss = 0.9472, Train Acc = 0.6491 | Val Loss = 0.9386, Val Acc = 0.6553
Epoch 005: Epoch Loss = 0.9429 | Train Loss = 0.9124, Train Acc = 0.6686 | Val Loss = 0.8990, Val Acc = 0.6768
Epoch 006: Epoch Loss = 0.9084 | Train Loss = 0.9053, Train Acc = 0.6649 | Val Loss = 0.8851, Val Acc = 0.6752
Epoch 007: Epoch Loss = 0.8696 | Train Loss = 0.8215, Train Acc = 0.6972 | Val Loss = 0.8218, Val Acc = 0.7038
Epoch 008: Epoch Loss = 0.8380 | Train Loss = 0.8120, Train Acc = 0.7008 | Val Loss = 0.8179, Val Acc = 0.7050
Epoch 009: Epoch Loss = 0.8065 | Train Loss = 0.8042, Train Acc = 0.6933 | Val Loss = 0.8054, Val Acc = 0.6977
E

test_accuracy,▁
test_loss,▁
test_accuracy,0.701
test_loss,0.85729


wandb: Agent Starting Run: 16bjwkmk with config:
wandb: 	activation_function: tanh
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.001
wandb: 	num_hidden_layers: 4
wandb: 	optimizer: nesterov
wandb: 	weight_decay: 0.0005
wandb: 	weight_initialisation: xavier


Epoch 001: Epoch Loss = 1.0199 | Train Loss = 0.6528, Train Acc = 0.7813 | Val Loss = 0.6484, Val Acc = 0.7830
Epoch 002: Epoch Loss = 0.5663 | Train Loss = 0.5054, Train Acc = 0.8253 | Val Loss = 0.5086, Val Acc = 0.8243
Epoch 003: Epoch Loss = 0.4842 | Train Loss = 0.4659, Train Acc = 0.8346 | Val Loss = 0.4757, Val Acc = 0.8273
Epoch 004: Epoch Loss = 0.4484 | Train Loss = 0.4345, Train Acc = 0.8486 | Val Loss = 0.4452, Val Acc = 0.8390
Epoch 005: Epoch Loss = 0.4253 | Train Loss = 0.4082, Train Acc = 0.8556 | Val Loss = 0.4210, Val Acc = 0.8477
Test Loss = 0.4501, Test Accuracy = 0.8378


test_accuracy,▁
test_loss,▁
test_accuracy,0.8378
test_loss,0.45008
